<h1 align="center">Exists2025:  Sexism Identification in Twitter</h1>
<h2 align="center">Task 1. Fine-tuning for binary classification
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

## Libraries

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback, AutoConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import random
import os
import pandas as pd
from readerEXIST2025 import EXISTReader
import re
import emoji
from create_submision_folder import create_submision_file

## Read data

In [2]:
url = r"\bhttps?://[\w.-]+(?:\.[a-zA-Z]{2,})?(?:/\S*/?)?"

emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
pattern_emoji = "(" + "|".join(re.escape(u) for u in emojis) + ")"

pattern_all = r"|".join(
    [
        url,
        pattern_emoji,
    ]
)

re_all = re.compile(pattern_all, re.U)

In [3]:
file_train = "data/EXIST 2025 Tweets Dataset/training/EXIST2025_training.json"
file_dev = "data/EXIST 2025 Tweets Dataset/dev/EXIST2025_dev.json"
file_test = "data/EXIST 2025 Tweets Dataset/test/EXIST2025_test_clean.json"

reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test, is_test=True)

EnTrainTask2, EnDevTask2, EnTestTask2 = reader_train.get(lang="EN", subtask="2", regular_exp=re_all, preprocess=True), reader_dev.get(lang="EN", subtask="2", regular_exp=re_all, preprocess=True), reader_test.get(lang="EN", subtask="2", regular_exp=re_all, preprocess=True)
SpTrainTask2, SpDevTask2, SpTestTask2 = reader_train.get(lang="ES", subtask="2", regular_exp=re_all, preprocess=True), reader_dev.get(lang="ES", subtask="2", regular_exp=re_all, preprocess=True), reader_test.get(lang="ES", subtask="2", regular_exp=re_all, preprocess=True)

## Set seed

In [4]:
def set_seed(seed=1234):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

## Dataset class

In [5]:
class SexismDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)
        }

## Metrics

In [6]:
def compute_metrics_2(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Pipeline

In [7]:
def sexism_classification_pipeline_task2(trainInfo, devInfo, testInfo=None, model_name='bert-base-uncased', nlabels=3, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc= LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype,
        ignore_mismatched_sizes=args.get("ignore_mismatched_sizes", False)
    )

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        #metric_for_best_model=args.get('metric_for_best_model',"ICM")
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_2,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    # If there is a test dataset
    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)

        })
        language = args.get("language", "english")
        submission_df.to_csv(f'sexism_predictions_task2_{language}.csv', index=False)
        print(f"Prediction TASK2 completed. Results saved to sexism_predictions_task2_{language}.csv")
        return model, submission_df
    return model, eval_results


## Training

In [8]:
modelname = "cardiffnlp/twitter-roberta-base-sentiment-latest"
params = {"num_train_epochs": 20,
          "learning_rate": 0.0001,
          "per_device_train_batch_size": 64,
          "warmup_steps": 200,
          "early_stopping_patience": 5,
          "ignore_mismatched_sizes": True,
          "logging_dir": None,
          "language": "english"
          }

_, eval_results = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, EnTestTask2, modelname, 3, "single_label_classification", **params)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.560700,1.226919,0.561644,0.241888,0.193853,0.321569
2,1.092100,0.919126,0.582192,0.245310,0.194064,0.333333
3,0.770400,0.852167,0.575342,0.406090,0.379593,0.483898
4,0.672900,0.747009,0.657534,0.484488,0.512808,0.484360
5,0.589700,0.800880,0.684932,0.514607,0.617295,0.533516
6,0.465900,0.931578,0.671233,0.524329,0.674932,0.568929
7,0.354800,1.107818,0.650685,0.524943,0.530051,0.531678
8,0.169600,1.320990,0.678082,0.577877,0.598403,0.572350
9,0.132100,1.671944,0.630137,0.498874,0.514478,0.516306
10,0.060800,1.681708,0.664384,0.567837,0.587384,0.564506


Validation Results: {'eval_loss': 1.3209902048110962, 'eval_accuracy': 0.678082191780822, 'eval_f1': 0.5778766526433637, 'eval_precision': 0.5984033887259694, 'eval_recall': 0.5723495458789576, 'eval_runtime': 0.7076, 'eval_samples_per_second': 206.328, 'eval_steps_per_second': 4.24, 'epoch': 13.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2_english.csv


In [9]:
modelname = "pysentimiento/robertuito-sentiment-analysis"
params = {"num_train_epochs": 20,
          "learning_rate": 0.0001,
          "per_device_train_batch_size": 64,
          "warmup_steps": 200,
          "early_stopping_patience": 5, 
          "ignore_mismatched_sizes": True,
          "logging_dir": None,
          "language": "spanish"
          }

_, eval_results = sexism_classification_pipeline_task2(SpTrainTask2, SpDevTask2, SpTestTask2, modelname, 3, "single_label_classification", **params)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.581100,1.401018,0.434783,0.264467,0.254775,0.287920
2,0.936100,1.003867,0.565217,0.336398,0.383417,0.378088
3,0.755200,0.878521,0.618357,0.408936,0.572435,0.429806
4,0.730400,0.775134,0.642512,0.527842,0.591051,0.513624
5,0.441600,0.800645,0.652174,0.535788,0.568417,0.526628
6,0.212800,0.834166,0.681159,0.585349,0.607963,0.574966
7,0.128700,1.047289,0.681159,0.604378,0.615100,0.602400
8,0.038900,1.284958,0.681159,0.593357,0.607875,0.587616
9,0.016600,1.829127,0.647343,0.522972,0.571458,0.512835
10,0.005600,1.674712,0.661836,0.596890,0.592813,0.605353


Validation Results: {'eval_loss': 1.0472886562347412, 'eval_accuracy': 0.6811594202898551, 'eval_f1': 0.604378013820074, 'eval_precision': 0.6150997150997152, 'eval_recall': 0.6024002704530088, 'eval_runtime': 0.9837, 'eval_samples_per_second': 210.422, 'eval_steps_per_second': 4.066, 'epoch': 12.0}
Prediction TASK2 completed. Results saved to sexism_predictions_task2_spanish.csv


## Save predictions

In [10]:
if create_submision_file("ArPa", 1, "hard", 1, 2, ".", "exist2025_ArPa"):
    print("Predictions saved")

Predictions saved
